# Web Scraping

Getting data from [statusinvest](https://statusinvest.com.br) stock market and creating visualizing the rank stocks to find the best ones according to indexes stipulated.

In [1]:
import requests
#from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

# Make a GET request to the webpage
url = 'https://statusinvest.com.br/acoes/busca-avancada'

driver = webdriver.Chrome()
driver.get(url)
sleep(1)

from selenium.webdriver.common.by import By

# Finding the elements
button_find = driver.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
sleep(1)

# # Close advertise
# button_adv = driver.find_element(By.XPATH, '/html/body/div[15]/div/div/div[1]/button').click()
# sleep(2)

# Download
button_download = driver.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a').click()

In [2]:
# Importing library
import pandas as pd

Path_name = '/Users/patriciasilva/Downloads/statusinvest-busca-avancada.csv'

df = pd.read_csv(Path_name, sep=';')

#df

# Replacing the null values for the zero
# Replace null values with zero
df.fillna('0', inplace=True)

#df.info(verbose=True, show_counts=True)

In [3]:
# Define the function to convert number notation and format to two decimal places
def convert_and_format(value):
    # Replace comma with dot and vice versa
    value = value.replace('.', '')
    value = value.replace(',', '.')
    # Convert string to float and format to two decimal places
    #value = '{:.2f}'.format(float(value))
    value = pd.to_numeric(value, errors='coerce')
    return value

# Apply the conversion and formatting function to the desired columns
columns_to_convert = ['PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO']
df[columns_to_convert] = df[columns_to_convert].applymap(convert_and_format)

# Convert the columns from string to numeric
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric)

# Print the modified DataFrame
#df

In [4]:
# Selecting the columns to analysis
df = df[['TICKER', 'DY', 'P/L', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 
       'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PASSIVOS / ATIVOS', 'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS',
       ' VPA', ' LPA']]

# Adding Valuation de Graham index:
df['VAL. GRAHAM'] = (22.5 * (df[' LPA'] * df[' VPA']))**(1/2)

df = df[['TICKER', 'DY', 'P/L', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 
       'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PASSIVOS / ATIVOS', 'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS',
       'VAL. GRAHAM']]

df

,TICKER,DY,P/L,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,LIQ. CORRENTE,ROE,ROA,ROIC,PASSIVOS / ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,VAL. GRAHAM
0,AALR3,0.00,-10.90,30.83,-1.55,-22.50,-158.48,-209.03,-50.55,0.95,0.63,-27.28,-9.62,-1.08,0.63,0.13,0.00,NaN
1,ABCB4,6.62,4.53,32.56,26.15,18.94,3.28,3.28,0.00,0.00,1.39,15.40,1.53,0.00,0.90,18.70,13.83,43.652474
2,ABEV3,5.28,15.44,49.71,22.46,18.03,12.39,11.91,-0.48,-0.10,0.99,17.35,10.88,19.66,0.36,10.72,15.19,10.686908
3,ADHM3,0.00,-4.09,100.00,-13548.48,-18860.61,-5.69,-5.69,0.00,0.00,0.00,-31.47,-2571.90,22.61,82.72,0.00,0.00,3.216442
4,AERI3,0.00,-10.29,13.11,9.22,-4.20,4.69,8.56,3.87,1.14,1.93,-13.47,-3.53,8.73,0.74,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,WIZC3,7.80,8.08,57.99,36.42,11.37,2.52,2.83,0.31,0.29,1.09,28.86,4.92,21.99,0.59,12.37,-6.64,6.256097
597,WLMM3,5.83,6.08,13.34,7.75,5.94,4.67,5.17,0.17,0.04,3.41,18.85,14.45,15.45,0.23,32.16,0.00,35.187647
598,WLMM4,5.66,6.89,13.34,7.75,5.94,5.28,5.17,0.17,0.04,3.41,18.85,14.45,15.45,0.23,32.16,0.00,35.187647
599,YDUQ3,0.00,264.48,57.39,15.09,0.31,5.44,11.36,5.93,1.35,1.74,0.47,0.15,7.87,0.67,6.20,-49.08,3.359129


In [6]:
# Making the analysis - finding the best stock according to indexes defined

# Add score based on conditions
df['Score'] = 0  # Initialize Score column with 0s

# Condition 1: Cell from Column1 > 8
DY = df['DY'] > 8
df.loc[DY, 'Score'] += 1

# Condition 2: Cell from Column2 < 10
PL = df['P/L'] < 10
df.loc[PL, 'Score'] += 1

# Condition 3: Cell from Column3 >= 15
Margem_Bruta = df['MARGEM BRUTA'] >= 15
df.loc[Margem_Bruta, 'Score'] += 1

# Condition 4: Cell from Column1 > 10
Margem_EBIT = df['MARGEM EBIT'] > 10
df.loc[Margem_EBIT, 'Score'] += 1

# Condition 5: Cell from Column2 > 10
Margem_LIQ = df['MARG. LIQUIDA'] > 10
df.loc[Margem_LIQ, 'Score'] += 1

# Condition 6: Cell from Column3 < 3
P_EBIT = df['P/EBIT'] < 3
df.loc[P_EBIT, 'Score'] += 1

# Condition 7: Cell from Column1 < 10
EV_EBIT = df['EV/EBIT'] < 10
df.loc[EV_EBIT, 'Score'] += 1

# Condition 8: Cell from Column2 < 2
DIV_LIQ_EBIT = df['DIVIDA LIQUIDA / EBIT'] < 2
df.loc[DIV_LIQ_EBIT, 'Score'] += 1

# Condition 9: Cell from Column3 < 1.5
DIV_LIQ_PATRI = df['DIV. LIQ. / PATRI.'] < 1.5
df.loc[DIV_LIQ_PATRI, 'Score'] += 1

# Condition 10: Cell from Column1 > 1
LIQ_Corrente = df['LIQ. CORRENTE'] > 1
df.loc[LIQ_Corrente, 'Score'] += 1

# Condition 11: Cell from Column2 > 10
ROE = df['ROE'] > 10
df.loc[ROE, 'Score'] += 1

# Condition 12: Cell from Column3 > 10
ROA = df['ROA'] > 10
df.loc[ROA, 'Score'] += 1

# Condition 13: Cell from Column1 > 10
ROIC = df['ROIC'] > 10
df.loc[ROIC, 'Score'] += 1

# Condition 14: Cell from Column2 < 1
Pass_ativos = df['PASSIVOS / ATIVOS'] < 1
df.loc[Pass_ativos, 'Score'] += 1

# Condition 15: Cell from Column3 > 10
CAGR_REC_FIVE = df['CAGR RECEITAS 5 ANOS'] > 10
df.loc[CAGR_REC_FIVE, 'Score'] += 1

# Condition 16: Cell from Column3 > 10
CAGR_LUC_FIVE = df['CAGR LUCROS 5 ANOS'] > 10
df.loc[CAGR_LUC_FIVE, 'Score'] += 1

# Sort by Score in descending order
df = df.sort_values('Score', ascending=False)

# Print the modified DataFrame
#df.head()

# Visualizing the top 50 stocks according the indexes defined
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.bar(df.head(50), x='TICKER', y='Score', barmode="group", labels={
                     "TICKER": "Stock",
                     "Score": "Score"}, title="Top 50 stocks according to defined scores")

fig